<a href="https://colab.research.google.com/github/korakot/pythainlp_workshop/blob/master/notebooks/children_day_motto_dek.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

เห็น [งานวิเคราะห์อันนี้](https://facebook.com/736738277/posts/10158040605708278) เลยอยากทำบ้าง

เอา data จาก [ข่าวสด](https://www.khaosod.co.th/special-stories/news_2076271)

## Import, install

In [0]:
import pandas as pd
from collections import Counter

In [0]:
%load_ext google.colab.data_table

In [3]:
!pip install --pre pythainlp

     |████████████████████████████████| 11.1MB 2.7MB/s 
     |████████████████████████████████| 1.5MB 41.6MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449905 sha256=5a3f4f2ed11c22e72cfa2463d0988e8d21732474d460604fcce4707f63af0fc5
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
Successfully built nltk
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.23.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
  Found existing installation: requests 2.21.0
    Uninstalling requests-2.21.0:
      Successfully uninstalled requests-2.21.0


In [0]:
import pythainlp
from pythainlp import word_tokenize, Tokenizer

In [0]:
from requests import get
from bs4 import BeautifulSoup

## Load data

In [9]:
url = 'https://www.khaosod.co.th/special-stories/news_2076271'
r = get(url)
soup = BeautifulSoup(r.text)
content = soup.find('div', 'udsg__content') #'td-post-content')
len(content.select('h3'))  # นายก 19 คน

19

In [8]:
soup

<!DOCTYPE html>
<html lang="th" prefix="og: http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0,maximum-scale=5.0" name="viewport"/>
<title>รวมมิตร คำขวัญวันเด็ก ทุกนายกฯ กว่า 60 ปี ผ่านตากันบ้างไหม จำได้รึเปล่าหนอ?</title><link href="https://www.khaosod.co.th/special-stories/news_2076271" rel="canonical"/><link href="https://www.khaosod.co.th/images/favicon.png" rel="icon" type="image/png"/><link href="https://www.khaosod.co.th/images/favicon.png" rel="apple-touch-icon"/><meta content="https://www.khaosod.co.th/special-stories/news_2076271" name="original-source"/><meta content="รวมมิตร คำขวัญวันเด็ก ทุกนายกฯ กว่า 60 ปี ผ่านตากันบ้างไหม จำได้รึเปล่าหนอ?" name="description"/><meta content="dm5rwarjo3n4ro0mo" name="dailymotion-domain-verification"/><meta content="ข่าวสด" name="author"/><meta content="1723786117864033" property="fb:app_id"/><link href="https://www.khaosod.co.th/wp/xmlrpc.php" rel="pingback"/>
<meta content="th_TH" prope

In [0]:
h3 = content.h3
h3

In [0]:
h3.find_next('ul')

In [0]:
li = h3.find_next('ul').li
li.text.split(maxsplit=3)

In [0]:
def gen_data():
  for h3 in content.select('h3'):
    pm = h3.text 
    for li in h3.find_next('ul').select('li'):
      _, year, _, motto = li.text.split(maxsplit=3)
      yield int(year), motto, pm

In [0]:
data = list(gen_data())
len(data)

In [0]:
pm = data[-1][-1]
pm

In [0]:
data.extend([
  (2562, 'เด็ก เยาวชน จิตอาสา ร่วมพัฒนาชาติ', pm),
  (2563, 'เด็กไทยยุคใหม่ รู้รักสามัคคี รู้หน้าที่พลเมืองไทย', pm),
])
len(data)

In [0]:
df = pd.DataFrame(data, columns='year motto pm'.split())
df

# Analyze

## Tokenizer

สามารถปรับได้เรื่อยๆ ถ้าเจอตัดผิด

In [0]:
from pythainlp.corpus import ttc
words = [w for w,_ in ttc.word_freqs()]
tok = Tokenizer(words)

## 9 คำฮิต

In [0]:
def gen_word():
  for motto in df.motto:
    yield from tok.word_tokenize(motto)

In [0]:
pairs = Counter(gen_word()).most_common(16)
stop = {' ', 'มี', 'ความ', 'เป็น', 'ของ'}
pd.DataFrame([(w, f) for (w, f) in pairs if w not in stop])
# นับตกไปบ้าง เพราะตัดไม่เหมือนกัน

## ใครบ่อยสุด

In [0]:
df.pm.value_counts().to_frame()

## ยาว, สั้น ที่สุด

In [0]:
l = df.motto.str.len()
df.loc[[l.idxmax(), l.idxmin()]]

In [0]:
len_w = df.motto.apply(lambda x: len(tok.word_tokenize(x)))
len_w.hist(bins=5, range=[0,25], grid=False);  # ทำไมรูปไม่เหมือน สงสัยตัดคำไม่เหมือน

In [0]:
len_w.describe()

## สิ่งแวดล้อม,​ เทคโนโลยี, ...

In [0]:
def contain(word):
  return df[df.motto.str.contains(word)]

In [0]:
contain('สิ่งแวดล้อม')

In [0]:
contain('เทคโนโลยี')

In [0]:
contain('ประชาธิปไตย')

In [0]:
contain('ปฏิวัติ')

# น่าทำต่อ

- แปลงเป็น vector แล้ว plot ให้ cluster เป็นกลุ่ม